# Connect To DB

In [ ]:
%load_ext sql
%sql mysql://no_pass@localhost:3306/HogwartzAccounting?local_infile=1

# Find Student Debt To University

In [ ]:
studentid = 41

In [ ]:
%%sql
SELECT

(
    -- amount owed by student
    SELECT SUM(COURSE_COST) FROM COURSE WHERE COURSE_ID IN (
        SELECT COURSE_ID FROM STUDENT_COURSE WHERE STUDENT_ID = {studentid}
    )
)
- 
(
    -- amount payed by student to university
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = {studentid} AND OUTGOING = 0
)
+ 
(
    -- amount payed by university to student
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = {studentid} AND OUTGOING = 1
) AS STUDENT_DEBT



# Find University Debt To Professor (Professor Credit)

In [ ]:
professor_name = '"Quill Annes"'

In [ ]:
%%sql
SELECT
prof.PERSON_ID as pid,
(
    -- amount owed by university
    SELECT SUM(crs.PROFESSOR_PAY * prof_crs.WEEKLY_HOURS) 
    FROM COURSE crs 
    JOIN PROFESSOR_COURSE prof_crs 
    ON crs.course_id = prof_crs.course_id
    WHERE prof_crs.PROFESSOR_ID = pid
)
- 
(
    -- amount payed by university to professor
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = pid AND OUTGOING = 1
)
+ 
(
    -- amount payed by professor to university
    SELECT SUM(AMOUNT) FROM BANK_TRANSFER WHERE PERSON_ID = pid AND OUTGOING = 0
) AS PROFESSOR_CREDIT
FROM PERSON per 
JOIN PROFESSOR prof 
ON per.person_id = prof.person_id 
WHERE CONCAT(per.first_name, ' ', per.last_name) = {professor_name}

# Find Courses Taught By "Old" Professors

In [ ]:
old_cutoff = '1991-01-01'

In [ ]:
%%sql
SELECT
    c.course_name, 
    CONCAT(per.first_name, ' ', per.last_name) AS professor_name, 
    per.date_of_birth
FROM 
    COURSE c
JOIN 
    PROFESSOR_COURSE pc ON c.course_id = pc.course_id
JOIN 
    PROFESSOR p ON pc.professor_id = p.person_id
JOIN 
    PERSON per ON p.person_id = per.person_id
WHERE 
    per.date_of_birth < "{old_cutoff}";

# Find How Profitable A Specific Course Is

In [ ]:
course_name = "Magical Geography"

In [ ]:
%%sql
SELECT 
    c.course_name,
    c.course_cost * COUNT(sc.student_id) AS total_course_revenue
FROM 
    COURSE c
JOIN 
    STUDENT_COURSE sc ON c.course_id = sc.course_id
WHERE 
    c.course_name = '{course_name}'
GROUP BY 
    c.course_name, c.course_cost;